# Разработка модели

В этой тетрадке вы должны создать свою модель и сохранить ее веса, чтобы потом их можно было загрузить уже из телеграм бота. Сейчас здесь находится бейзлайн, который вам нужно адаптировать под свой датасет и улучшить модель под него. Так как вы скорее всего будете использовать предобученные модели, то улучшение заключается в нахождении лучшей из них и правильном методе тренировки.

In [0]:
!pip install -q Pillow==4.1.1
!pip install -q "fastai==0.7.0"
!pip install -q torchtext==0.2.3
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python

from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
!apt update && apt install -y libsm6 libxext6

import os
accelerator = 'cu80' if os.path.exists('/opt/bin/nvidia-smi') else 'cpu'
!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
!pip install -q image

In [0]:
from __future__ import print_function, division
import time

import pathlib
import gc
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline   
from PIL import Image
import shutil
import pickle
from skimage import io
from tqdm import tqdm, tqdm_notebook
from pathlib import Path
from multiprocessing.pool import ThreadPool
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings(action='ignore', category=DeprecationWarning)
from sklearn.model_selection import train_test_split
from imageio import imread
from skimage.transform import resize
from skimage import color
from sklearn.metrics import confusion_matrix

plt.ion()

# В этой части нужно загрузить датасет, разделить его на тренировочную и валидационную выборки и разложить картинки по папкам, чтобы в одной папке лежали картинки одного класса.

Эта часть очень сильно зависит от датасета, который вы выбрали, поэтому включить ее в бейзлайн не получится. Если вы выберете датасет, но застрянете на этом шаге, то напишите мне и я помогу разобраться. Это очень важно и не стоит откладывать, так как без приведения датасета к удобному формату вы не сможете продолжить проект.

In [0]:
PATH = '..flowers/' # линк к картинкам на гугл-диске
sz=224


Теперь посмотрим на картинки.

In [0]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.figure(figsize=(15, 12))
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)


# Получим 1 батч (картнки-метки) из обучающей выборки
inputs, classes = next(iter(dataloaders['train']))

# Расположим картинки рядом
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

Теперь определим функции для тренировки модели и ее оценки.

In [0]:
def visualize_model(model, num_images=6):
    images_so_far = 0
    fig = plt.figure()

    for i, data in enumerate(dataloaders['val']):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)

        for j in range(inputs.size()[0]):
            images_so_far += 1
            ax = plt.subplot(num_images // 2, 2, images_so_far)
            ax.axis('off')
            ax.set_title('predicted: {}'.format(class_names[preds[j]]))
            imshow(inputs.cpu().data[j])

            if images_so_far == num_images:
                return

Определим саму модель. При тренировке мы бд

In [0]:
# выбор модели
arch = resnet34 

In [0]:
# загрузка данных
data = ImageClassifierData.from_paths(PATH, tfms=tfms_from_model(arch, sz))
data.path = pathlib.Path('.')  

In [0]:
# запуск обучения
learn = ConvLearner.pretrained(arch, data, precompute=False) 
learn.precompute = False
learn.unfreeze()
lr=np.array([1e-4,1e-3,1e-2])
learn.fit(lr, 5, cycle_len=1, cycle_mult=2)

In [0]:
# сохраняем модель
torch.save(learn.model.state_dict(), "model_for_bot.h5")